In [56]:
#############################################################################
### This script computes various network metrics and provides Visualization from CSV files containing
### weighted adjacency matrices with coordinates. 
### DEBUGGED AND EDITED
#############################################################################

In [3]:
import numpy as np
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
import seaborn as sns
from collections import defaultdict, Counter
import community.community_louvain as community_louvain
import community as community_louvain
import os, glob


# Set plotting style
plt.style.use('ggplot')
sns.set_context("notebook", font_scale=1.5)


In [4]:
#############################################################################################
# Define the directory and file patterns
# Adjust the paths as necessary for your environment
#############################################################################################

data_dir = r"C:\Users\hp.LAPTOP-OG98VQR6\OneDrive\Desktop\ERSA\Agroforestal\Discover Networks\Simulations\Simulation6"
# file_path =  r"C:\Users\hp.LAPTOP-OG98VQR6\OneDrive\Desktop\ERSA\Agroforestal\Discover Networks\Simulations\Simulation5\weighted_matrix_combined_tick_338.csv"
# file_path =  r"C:\Users\hp.LAPTOP-OG98VQR6\OneDrive\Desktop\ERSA\Agroforestal\Discover Networks\Simulations\Simulation5\weighted_matrix_with_coords_combined_tick_338.csv"

file_path = r"C:\Users\hp.LAPTOP-OG98VQR6\OneDrive\Desktop\ERSA\Agroforestal\Discover Networks\Simulations\Simulation6\weighted_matrix_combined_tick_338.csv"

data_dir = r"C:\Users\hp.LAPTOP-OG98VQR6\OneDrive\Desktop\ERSA\Agroforestal\Discover Networks\Simulations\Simulation6"
pattern     = os.path.join(data_dir, "weighted_matrix_xy_combined_tick_*.csv")
output_dir = os.path.join(data_dir, "Metrics_Results")
os.makedirs(output_dir, exist_ok=True)
file_path = r"C:\Users\hp.LAPTOP-OG98VQR6\OneDrive\Desktop\ERSA\Agroforestal\Discover Networks\Simulations\Simulation6\weighted_matrix_with_coords_combined_tick_182.csv"

pattern = os.path.join(data_dir, "weighted_matrix_with_coords_combined_tick_*.csv")
files = glob.glob(pattern)
# sort by tick-number at end of filename
files = sorted(files, key=lambda fp: int(os.path.basename(fp).split('_')[-1].split('.')[0]))

In [5]:
############################################################################################
# Load network from CSV file
############################################################################################

# Load network from CSV file
def load_network_from_csv(file_path):
    """
    Load network data from CSV file into a NetworkX graph
    """
    print(f"Loading network from: {file_path}")
    # Read the CSV file
    df = pd.read_csv(file_path)
    print(f"Loaded data with {len(df)} rows")
    
    # Display first few rows to verify structure
    print("\nFirst few rows of data:")
    print(df.head())
    
    # Create an empty directed graph
    G = nx.DiGraph()
    
    # Add nodes and edges to the graph
    for _, row in df.iterrows():
        source = row['source']
        target = row['target']
        weight = row['weight']
        
        # Add nodes if they don't exist
        if not G.has_node(source):
            G.add_node(source)
        if not G.has_node(target):
            G.add_node(target)
        
        # Add edge with weight
        G.add_edge(source, target, weight=weight)
    
    print(f"Created graph with {G.number_of_nodes()} nodes and {G.number_of_edges()} edges")
    return G


In [6]:
############################################################################################
# Load network from CSV file
############################################################################################

def load_network_csv(file_path):
    """
    Load network data from CSV file into a NetworkX graph,
    dropping any rows whose weight cannot be parsed as a float.
    """
    df = pd.read_csv(file_path)

    # coerce weight to numeric, setting invalid parses to NaN
    df['weight'] = pd.to_numeric(df['weight'], errors='coerce')

    # drop any rows where source/target aren't integers or weight is NaN
    df = df.dropna(subset=['source','target','weight'])
    df['source'] = df['source'].astype(int)
    df['target'] = df['target'].astype(int)
    df['weight'] = df['weight'].astype(float)

    G = nx.DiGraph()
    for _, row in df.iterrows():
        G.add_edge(row['source'],
                   row['target'],
                   weight=row['weight'],
                   label=row.get('label',None))
    return G

In [7]:
############################################################################################
# Analyze direct interactions (1-hop)
############################################################################################

def analyze_direct_and_indirect_interactions(G):
    """
    Analyze direct (1-hop) and indirect (2-hop) interactions in an ecological network
    where weights represent interaction strength (higher = stronger interaction)
    and signs represent interaction type (positive = synergy, negative = antagonism)
    """
    results = {}
    
    # 1. Basic network properties
    results['num_nodes'] = G.number_of_nodes()
    results['num_edges'] = G.number_of_edges()
    results['density'] = nx.density(G)
    
    # 2. Direct interaction analysis
    direct_interactions = analyze_direct_interactions(G)
    results.update(direct_interactions)
    
    # 3. Indirect interaction analysis (2-hop paths)
    indirect_interactions = analyze_indirect_interactions(G)
    results.update(indirect_interactions)
    
    # 4. Node-level metrics
    node_metrics = compute_node_level_metrics(G)
    results['node_metrics'] = node_metrics
    
    # 5. Community detection based on interaction patterns
    communities = detect_interaction_communities(G)
    results['communities'] = communities
    
    return results

In [8]:
############################################################################################
# Analyze direct interactions (1-hop)
############################################################################################

def analyze_direct_interactions(G):
    """
    Analyze direct (1-hop) interactions between species
    """
    metrics = {}
    
    # Count positive and negative interactions
    pos_edges = [(u, v) for u, v, w in G.edges(data='weight') if w > 0]
    neg_edges = [(u, v) for u, v, w in G.edges(data='weight') if w < 0]
    
    metrics['positive_edges'] = len(pos_edges)
    metrics['negative_edges'] = len(neg_edges)
    metrics['pos_neg_ratio'] = len(pos_edges) / len(neg_edges) if len(neg_edges) > 0 else float('inf')
    
    # Calculate average strength of positive and negative interactions
    pos_weights = [G[u][v]['weight'] for u, v in pos_edges]
    neg_weights = [abs(G[u][v]['weight']) for u, v in neg_edges]
    
    metrics['avg_positive_strength'] = np.mean(pos_weights) if pos_weights else 0
    metrics['avg_negative_strength'] = np.mean(neg_weights) if neg_weights else 0
    metrics['max_positive_strength'] = max(pos_weights) if pos_weights else 0
    metrics['max_negative_strength'] = max(neg_weights) if neg_weights else 0
    
    # Calculate reciprocity (proportion of interactions that are reciprocated)
    metrics['reciprocity'] = nx.reciprocity(G)
    
    # Calculate asymmetry in interactions
    asymmetry_scores = []
    for u, v in G.edges():
        if G.has_edge(v, u):
            w1 = G[u][v]['weight']
            w2 = G[v][u]['weight']
            # Measure both strength asymmetry and sign asymmetry
            strength_asymmetry = abs(abs(w1) - abs(w2)) / (abs(w1) + abs(w2)) if (abs(w1) + abs(w2)) > 0 else 0
            sign_different = (w1 * w2) < 0  # True if signs are different
            asymmetry_scores.append((strength_asymmetry, sign_different))
    
    if asymmetry_scores:
        metrics['avg_strength_asymmetry'] = np.mean([a[0] for a in asymmetry_scores])
        metrics['prop_sign_differences'] = np.mean([int(a[1]) for a in asymmetry_scores])
    else:
        metrics['avg_strength_asymmetry'] = 0
        metrics['prop_sign_differences'] = 0
    
    return metrics

In [9]:
############################################################################################
# Analyze indirect interactions (1-hop)
############################################################################################

def analyze_indirect_interactions(G):
    """
    Analyze indirect (2-hop) interactions and their relationship to direct interactions
    """
    metrics = {}
    
    # Store all 2-hop paths and their compound effect
    two_hop_paths = []
    two_hop_effects = []
    two_hop_connections = defaultdict(list)
    
    # For each pair of nodes
    for source in G.nodes():
        for target in G.nodes():
            if source != target:
                # Check if there's a direct connection
                direct_effect = G[source][target]['weight'] if G.has_edge(source, target) else 0
                
                # Find all 2-hop paths (through one intermediate node)
                indirect_paths = []
                for intermediate in G.successors(source):
                    if G.has_edge(intermediate, target) and intermediate != target:
                        weight1 = G[source][intermediate]['weight']
                        weight2 = G[intermediate][target]['weight']
                        compound_effect = weight1 * weight2  # Multiplication preserves sign logic
                        
                        indirect_paths.append({
                            'path': (source, intermediate, target),
                            'effect': compound_effect,
                            'weights': (weight1, weight2)
                        })
                
                if indirect_paths:
                    # Store all indirect paths
                    two_hop_paths.extend(indirect_paths)
                    
                    # Calculate total indirect effect (sum of all 2-hop paths)
                    total_indirect_effect = sum(p['effect'] for p in indirect_paths)
                    two_hop_effects.append(total_indirect_effect)
                    
                    # Compare direct and indirect effects if both exist
                    if direct_effect != 0:
                        two_hop_connections[(source, target)].append({
                            'direct_effect': direct_effect,
                            'indirect_effect': total_indirect_effect,
                            'indirect_paths': indirect_paths
                        })
    
    # Analyze the distribution of 2-hop effects
    if two_hop_effects:
        metrics['num_2hop_paths'] = len(two_hop_paths)
        metrics['avg_2hop_effect'] = np.mean(two_hop_effects)
        metrics['max_2hop_effect'] = max(two_hop_effects)
        metrics['min_2hop_effect'] = min(two_hop_effects)
        
        # Count positive and negative 2-hop effects
        pos_2hop = sum(1 for e in two_hop_effects if e > 0)
        neg_2hop = sum(1 for e in two_hop_effects if e < 0)
        metrics['positive_2hop'] = pos_2hop
        metrics['negative_2hop'] = neg_2hop
        metrics['pos_neg_2hop_ratio'] = pos_2hop / neg_2hop if neg_2hop > 0 else float('inf')
    
    # Analyze reinforcing vs. counteracting effects
    if two_hop_connections:
        reinforcing = 0
        counteracting = 0
        total = 0
        
        for (source, target), connections in two_hop_connections.items():
            for conn in connections:
                direct = conn['direct_effect']
                indirect = conn['indirect_effect']
                
                if direct * indirect > 0:  # Same sign - reinforcing
                    reinforcing += 1
                else:  # Different sign - counteracting
                    counteracting += 1
                total += 1
        
        metrics['reinforcing_effects'] = reinforcing
        metrics['counteracting_effects'] = counteracting
        metrics['reinforcing_ratio'] = reinforcing / total if total > 0 else 0
    
    return metrics


In [10]:
############################################################################################
# Compute node-level metrics
############################################################################################

def ecological_closeness_centrality(G):
    """
    Custom ecological closeness centrality that:
    1. Treats stronger interactions as "closer"
    2. Preserves sign information 
    3. Only considers 1-hop and 2-hop neighbors
    """
    closeness_scores = {}
    
    for node in G.nodes():
        # Direct interactions (1-hop)
        direct_neighbors = list(G.successors(node))
        direct_influences = {}
        
        for neighbor in direct_neighbors:
            weight = G[node][neighbor]['weight']
            # Transform weight: stronger interactions (higher |weight|) mean closer ecological distance
            # We use absolute value but preserve sign with signum function
            influence = np.sign(weight) * (1.0 / (1.0 + np.exp(-abs(weight))))
            direct_influences[neighbor] = influence
        
        # Indirect interactions (2-hop)
        indirect_influences = {}
        for direct_neighbor in direct_neighbors:
            for indirect_neighbor in G.successors(direct_neighbor):
                if indirect_neighbor == node or indirect_neighbor in direct_neighbors:
                    continue  # Skip self-loops and direct neighbors
                
                weight1 = G[node][direct_neighbor]['weight']
                weight2 = G[direct_neighbor][indirect_neighbor]['weight']
                
                # Compound effect with attenuation
                compound_effect = np.sign(weight1 * weight2) * (abs(weight1 * weight2) ** 0.5)
                
                # Dampen effect based on path length (2-hop gets 50% weight)
                influence = 0.5 * np.sign(compound_effect) * (1.0 / (1.0 + np.exp(-abs(compound_effect))))
                
                # Add to existing influence if there are multiple paths
                if indirect_neighbor in indirect_influences:
                    indirect_influences[indirect_neighbor] += influence
                else:
                    indirect_influences[indirect_neighbor] = influence
        
        # Calculate positive and negative influence reaches
        pos_direct = sum(infl for infl in direct_influences.values() if infl > 0)
        neg_direct = sum(abs(infl) for infl in direct_influences.values() if infl < 0)
        pos_indirect = sum(infl for infl in indirect_influences.values() if infl > 0)
        neg_indirect = sum(abs(infl) for infl in indirect_influences.values() if infl < 0)
        
        # Calculate total ecological influence (closeness)
        total_positive = pos_direct + pos_indirect
        total_negative = neg_direct + neg_indirect
        total_influence = total_positive + total_negative
        net_influence = total_positive - total_negative
        
        closeness_scores[node] = {
            'positive_influence': total_positive,
            'negative_influence': total_negative,
            'total_influence': total_influence,
            'net_influence': net_influence,
            'direct_reach': len(direct_influences),
            'indirect_reach': len(indirect_influences),
            'total_reach': len(direct_influences) + len(indirect_influences)
        }
    
    return closeness_scores

In [14]:
############################################################################################
# Compute node-level metrics
############################################################################################

def compute_node_level_metrics(G):
    """
    Compute various node-level ecological metrics
    """
    node_metrics = {}
    
    # Calculate ecological closeness centrality
    ecological_closeness = ecological_closeness_centrality(G)
    node_metrics['ecological_closeness'] = ecological_closeness
    
    # Calculate number of positive and negative connections per node
    for node in G.nodes():
        pos_out = sum(1 for _, v, w in G.out_edges(node, data='weight') if w > 0)
        neg_out = sum(1 for _, v, w in G.out_edges(node, data='weight') if w < 0)
        pos_in = sum(1 for u, _, w in G.in_edges(node, data='weight') if w > 0)
        neg_in = sum(1 for u, _, w in G.in_edges(node, data='weight') if w < 0)
        
        if node not in node_metrics:
            node_metrics[node] = {}
            
        node_metrics[node]['positive_outgoing'] = pos_out
        node_metrics[node]['negative_outgoing'] = neg_out
        node_metrics[node]['positive_incoming'] = pos_in
        node_metrics[node]['negative_incoming'] = neg_in
        node_metrics[node]['net_positive'] = (pos_out + pos_in) - (neg_out + neg_in)
    
    # Identify keystone species (high betweenness in 2-hop paths)
    keystone_scores = {}
    for node in G.nodes():
        # Count how many 2-hop paths go through this node
        path_count = 0
        for source in G.nodes():
            if source == node:
                continue
            for target in G.nodes():
                if target == node or target == source:
                    continue
                if G.has_edge(source, node) and G.has_edge(node, target):
                    path_count += 1
        
        keystone_scores[node] = path_count
    
    # Normalize keystone scores
    max_score = max(keystone_scores.values()) if keystone_scores else 1
    for node, score in keystone_scores.items():
        if node not in node_metrics:
            node_metrics[node] = {}
        node_metrics[node]['keystone_score'] = score / max_score if max_score > 0 else 0
    
    return node_metrics

In [13]:
############################################################################################
# Detect communities based on interaction patterns
############################################################################################

def detect_interaction_communities(G):
    """
    Detect communities based on interaction patterns
    """
    # Create a modified graph for community detection where:
    # - Edge weights are converted to represent similarity (higher = more similar)
    # - Negative edges represent dissimilarity
    G_comm = nx.Graph()
    
    for u, v, data in G.edges(data=True):
        weight = data['weight']
        # Convert weight to similarity measure: |weight| represents strength, sign preserved
        if weight > 0:  # Positive interaction = similarity
            sim_weight = weight
        else:  # Negative interaction = dissimilarity
            sim_weight = weight  # Keep negative to represent dissimilarity
        
        if G_comm.has_edge(u, v):
            # If edge exists, add to current weight (accumulate evidence)
            G_comm[u][v]['weight'] += sim_weight
        else:
            G_comm.add_edge(u, v, weight=sim_weight)
    
    # Create a positive-only graph for community detection algorithms that require it
    G_pos = nx.Graph()
    for u, v, data in G_comm.edges(data=True):
        # Take absolute weight but mark edges that were negative
        weight = abs(data['weight'])
        is_negative = data['weight'] < 0
        G_pos.add_edge(u, v, weight=weight, is_negative=is_negative)
    
    # Detect communities using Louvain method
    try:
        # Check if python-louvain is installed
        import community as community_louvain
        partition = community_louvain.best_partition(G_pos, weight='weight')
        
        # Convert partition format
        communities = defaultdict(list)
        for node, community_id in partition.items():
            communities[community_id].append(node)
        
        return {
            'algorithm': 'louvain',
            'communities': dict(communities),
            'num_communities': len(communities)
        }
    except ImportError:
        # Fall back to NetworkX's community detection if python-louvain is not available
        try:
            from networkx.algorithms import community
            communities_generator = community.greedy_modularity_communities(G_pos, weight='weight')
            communities = {i: list(comm) for i, comm in enumerate(communities_generator)}
            
            return {
                'algorithm': 'greedy_modularity',
                'communities': communities,
                'num_communities': len(communities)
            }
        except Exception as e:
            return {
                'algorithm': 'failed',
                'error': str(e),
                'communities': {},
                'num_communities': 0
            }

In [16]:
############################################################################################    
# Visualize the ecological network using actual spatial coordinates
############################################################################################

def visualize_spatial_network(G, node_coords, node_metrics=None, communities=None, 
                             title="Spatial Ecological Network", pos_color='blue', 
                             neg_color='red', max_edges=2000, node_size_factor=100,
                             edge_width_factor=1.0, show_labels=False):
    """
    Visualize the ecological network using actual spatial coordinates
    """
    # Create figure and axis explicitly
    fig, ax = plt.subplots(figsize=(16, 16))
    
    # Use the actual spatial coordinates for node positions
    pos = node_coords
    
    # Prepare node sizes based on metrics or default size
    if node_metrics:
        node_sizes = []
        for node in G.nodes():
            if node in node_metrics and 'ecological_closeness' in node_metrics[node]:
                # Size based on total influence
                influence = node_metrics[node]['ecological_closeness'].get('total_influence', 0)
                size = node_size_factor * (1 + influence)
            else:
                # Fallback size
                size = node_size_factor
            node_sizes.append(size)
    else:
        # Default node size
        node_sizes = [node_size_factor] * G.number_of_nodes()
    
    # Prepare node colors based on communities
    if communities and 'communities' in communities and communities['communities']:
        # Create a reverse mapping from node to community ID
        node_community = {}
        for comm_id, nodes in communities['communities'].items():
            for node in nodes:
                node_community[node] = int(comm_id)
        
        # Generate colors for nodes
        node_colors = [node_community.get(node, 0) for node in G.nodes()]
        # Use a qualitative colormap
        import matplotlib as mpl
        max_community = max(node_community.values()) if node_community else 0
        cmap = mpl.colormaps.get_cmap('tab20')
        norm = mpl.colors.Normalize(vmin=0, vmax=max_community)
    else:
        # Default coloring - GREEN nodes as requested
        node_colors = 'green'
        cmap = None
        norm = None
    
    # Separate positive and negative edges
    pos_edges = [(u, v) for u, v, data in G.edges(data=True) if data['weight'] > 0]
    neg_edges = [(u, v) for u, v, data in G.edges(data=True) if data['weight'] < 0]
    
    # Limit the number of edges for large networks
    if len(pos_edges) > max_edges:
        import random
        random.seed(42)
        pos_edges = random.sample(pos_edges, max_edges)
    if len(neg_edges) > max_edges:
        import random
        random.seed(42)
        neg_edges = random.sample(neg_edges, max_edges)
    
    # Calculate edge widths based on absolute weight
    pos_weights = [edge_width_factor * abs(G[u][v]['weight']) for u, v in pos_edges]
    neg_weights = [edge_width_factor * abs(G[u][v]['weight']) for u, v in neg_edges]
    
    # Draw nodes - GREEN as requested
    nodes = nx.draw_networkx_nodes(G, pos, ax=ax, node_size=node_sizes, node_color=node_colors, 
                                  alpha=0.7, cmap=cmap)
    
    # Draw edges - BLUE for positive, RED for negative as requested
    nx.draw_networkx_edges(G, pos, ax=ax, edgelist=pos_edges, width=pos_weights, 
                          edge_color=pos_color, alpha=0.6, arrows=True, arrowsize=10)
    nx.draw_networkx_edges(G, pos, ax=ax, edgelist=neg_edges, width=neg_weights, 
                          edge_color=neg_color, alpha=0.6, arrows=True, 
                          arrowstyle='-|>', arrowsize=15)
    
    # Add labels if requested
    if show_labels:
        # Only label nodes with significant importance
        if node_metrics:
            important_nodes = {}
            for node in G.nodes():
                if (node in node_metrics and 
                    'ecological_closeness' in node_metrics[node] and
                    node_metrics[node]['ecological_closeness'].get('total_influence', 0) > 0.2):
                    important_nodes[node] = node
                    
            nx.draw_networkx_labels(G, pos, ax=ax, labels=important_nodes, font_size=8)
        else:
            nx.draw_networkx_labels(G, pos, ax=ax, font_size=8)
    
    ax.set_title(title, fontsize=16)
    
    # Set the axes to reflect actual spatial coordinates
    if node_coords:
        x_coords = [coord[0] for coord in node_coords.values()]
        y_coords = [coord[1] for coord in node_coords.values()]
        ax.set_xlim(min(x_coords) - 5, max(x_coords) + 5)
        ax.set_ylim(min(y_coords) - 5, max(y_coords) + 5)
    
    # Set equal aspect ratio to maintain spatial integrity
    ax.set_aspect('equal', 'box')
    
    # Turn off axis
    ax.axis('off')
    
    # Add colorbar for community colors if available - FIXED VERSION
    if communities and communities['communities'] and cmap and norm:
        sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
        sm.set_array([])
        # Fix: Specify the axes for the colorbar
        cbar = plt.colorbar(sm, ax=ax)
        cbar.set_label('Community')
    
    plt.tight_layout()
    return fig, ax


In [17]:
############################################################################################
# Analyze direct interactions (1-hop)
############################################################################################

def analyze_direct_and_indirect_interactions(G):
    """
    Analyze direct (1-hop) and indirect (2-hop) interactions in an ecological network
    where weights represent interaction strength (higher = stronger interaction)
    and signs represent interaction type (positive = synergy, negative = antagonism)
    """
    results = {}
    
    # 1. Basic network properties
    results['num_nodes'] = G.number_of_nodes()
    results['num_edges'] = G.number_of_edges()
    results['density'] = nx.density(G)
    
    # 2. Direct interaction analysis
    direct_interactions = analyze_direct_interactions(G)
    results.update(direct_interactions)
    
    # 3. Indirect interaction analysis (2-hop paths)
    indirect_interactions = analyze_indirect_interactions(G)
    results.update(indirect_interactions)
    
    # 4. Node-level metrics
    node_metrics = compute_node_level_metrics(G)
    results['node_metrics'] = node_metrics
    
    # 5. Community detection based on interaction patterns
    communities = detect_interaction_communities(G)
    results['communities'] = communities
    
    return results

In [18]:
############################################################################################
# Summarize ecological metrics in a human-readable format
############################################################################################

def summarize_ecological_metrics(metrics):
    """
    Generate a human-readable summary of the ecological metrics
    """
    summary = []
    
    # Network overview
    summary.append(f"### Ecological Network Summary")
    summary.append(f"Network size: {metrics['num_nodes']} species with {metrics['num_edges']} interactions")
    summary.append(f"Network density: {metrics['density']:.4f}")
    
    # Direct interactions
    summary.append(f"\n### Direct Interactions")
    summary.append(f"Positive interactions: {metrics['positive_edges']} ({metrics['avg_positive_strength']:.4f} avg strength)")
    summary.append(f"Negative interactions: {metrics['negative_edges']} ({metrics['avg_negative_strength']:.4f} avg strength)")
    summary.append(f"Positive-to-negative ratio: {metrics['pos_neg_ratio']:.4f}")
    summary.append(f"Reciprocity: {metrics['reciprocity']:.4f}")
    
    # Indirect interactions
    if 'num_2hop_paths' in metrics:
        summary.append(f"\n### Indirect (2-hop) Interactions")
        summary.append(f"Total 2-hop paths: {metrics['num_2hop_paths']}")
        summary.append(f"Positive indirect effects: {metrics['positive_2hop']}")
        summary.append(f"Negative indirect effects: {metrics['negative_2hop']}")
        
        if 'reinforcing_effects' in metrics:
            summary.append(f"Reinforcing effects: {metrics['reinforcing_effects']} paths")
            summary.append(f"Counteracting effects: {metrics['counteracting_effects']} paths")
            summary.append(f"Reinforcement ratio: {metrics['reinforcing_ratio']:.4f}")
    
    # Community structure
    if 'communities' in metrics and 'num_communities' in metrics['communities']:
        summary.append(f"\n### Community Structure")
        summary.append(f"Number of communities: {metrics['communities']['num_communities']}")
        summary.append(f"Community detection algorithm: {metrics['communities']['algorithm']}")
        
        # List community sizes
        if metrics['communities']['communities']:
            comm_sizes = [len(nodes) for comm_id, nodes in metrics['communities']['communities'].items()]
            summary.append(f"Community sizes: min={min(comm_sizes)}, max={max(comm_sizes)}, avg={np.mean(comm_sizes):.1f}")
    
    # Node-level metrics
    if 'node_metrics' in metrics:
        # Find top species by different metrics
        summary.append(f"\n### Key Species")
        
        # Try to identify keystone species (high keystone score)
        keystone_scores = {node: data.get('keystone_score', 0) 
                          for node, data in metrics['node_metrics'].items() 
                          if isinstance(data, dict) and 'keystone_score' in data}
        
        if keystone_scores:
            top_keystones = sorted(keystone_scores.items(), key=lambda x: x[1], reverse=True)[:5]
            summary.append(f"Top keystone species (highest intermediary importance):")
            for i, (species, score) in enumerate(top_keystones, 1):
                summary.append(f"  {i}. Species {species}: score={score:.4f}")
        
        # Species with highest total influence
        closeness_scores = {node: data.get('ecological_closeness', {}).get('total_influence', 0) 
                           for node, data in metrics['node_metrics'].items() 
                           if isinstance(data, dict) and 'ecological_closeness' in data}
        
        if closeness_scores:
            top_influential = sorted(closeness_scores.items(), key=lambda x: x[1], reverse=True)[:5]
            summary.append(f"Most influential species (highest total ecological impact):")
            for i, (species, score) in enumerate(top_influential, 1):
                summary.append(f"  {i}. Species {species}: impact={score:.4f}")
    
    return "\n".join(summary)


In [ ]:
############################################################################################
### EXAMPLES
############################################################################################

# Example usage
# file_path = "weighted_matrix_combined_tick_182.csv"
# G = load_network_from_csv(file_path)  # Using the function from your original code
# 
# # Analyze ecological metrics
# metrics = analyze_direct_and_indirect_interactions(G)
# 
# # Print summary
# print(summarize_ecological_metrics(metrics))
# 
# # Visualize ecological network
# visualize_ecological_network(G, 
#                            node_metrics=metrics['node_metrics'],
#                            communities=metrics['communities'],
#                            title=f"Species Interaction Network (Communities)")
# plt.savefig("ecological_network_communities.png", dpi=300)
# plt.show()

In [43]:
file_path = r"C:\Users\hp.LAPTOP-OG98VQR6\OneDrive\Desktop\ERSA\Agroforestal\Discover Networks\Simulations\Simulation6\weighted_matrix_with_coords_combined_tick_182.csv"

data_dir = r"C:\Users\hp.LAPTOP-OG98VQR6\OneDrive\Desktop\ERSA\Agroforestal\Discover Networks\Simulations\Simulation6"
pattern     = os.path.join(data_dir, "weighted_matrix_xy_combined_tick_*.csv")
files = glob.glob(pattern)


In [ ]:
############################################################################################
# Load the spatial coordinates from a CSV file and create a spatial graph
############################################################################################

# Load your network
G = load_network_from_csv(file_path)

# Extract node coordinates from the CSV file
df = pd.read_csv(file_path)
coord_rows = pd.concat([
    df[['source','source_x','source_y']].rename(columns={'source':'node','source_x':'x','source_y':'y'}),
    df[['target','target_x','target_y']].rename(columns={'target':'node','target_x':'x','target_y':'y'})
]).drop_duplicates(subset='node')
pos = { int(r.node): (r.x, r.y) for r in coord_rows.itertuples() }

# Ensure analyze_direct_interactions is defined (run CELL INDEX: 6 if not)
metrics = analyze_direct_and_indirect_interactions(G)

# Print a human-readable summary
print(summarize_ecological_metrics(metrics))

# Visualize the network with ecological interpretations
visualize_spatial_network(G, pos, 
                         node_metrics=metrics['node_metrics'],
                         communities=metrics['communities'],
                         title="Spatial Ecological Network",
                         edge_width_factor=2.0,
                         node_size_factor=50,
                         show_labels=True)
fig, ax = visualize_spatial_network(G, pos, 
                         node_metrics=metrics['node_metrics'],
                         communities=metrics['communities'],
                         title="Spatial Ecological Network",
                         edge_width_factor=2.0,
                         node_size_factor=50,
                         show_labels=True)

# Extract tick number from file_path
tick = int(os.path.basename(file_path).split('_')[-1].split('.')[0])

ax.set_title(f"Spatial Ecological Network at tick {tick}")  # if you want to override
outname = os.path.join(output_dir, f"Spatial_Ecological_Network_{tick:03d}.png")
fig.savefig(outname, dpi=300, bbox_inches='tight')
plt.close(fig)
os.makedirs(output_dir, exist_ok=True)

# draw → get the figure that the function creates
fig = visualize_spatial_network(
    G, pos,
    node_metrics=metrics['node_metrics'],
    communities=metrics['communities'],
    title=f"Network at tick {tick}",
    edge_width_factor=2.0,
    node_size_factor=50,
    show_labels=True
)

# save THAT figure
outname = os.path.join(output_dir, f"Spatial_Ecological_Network_{tick:03d}.png")
fig.savefig(outname, dpi=300, bbox_inches='tight')
print(f"Saved: {outname} (exists: {os.path.exists(outname)})")
plt.show()  # show the figure
plt.close(fig)  # free memory when looping many ticks


In [51]:
############################################################################################
# Analyze spatial information structure
############################################################################################

# After loading your network and computing metrics
info_analysis = analyze_spatial_information_structure(
    G, pos, metrics['node_metrics'], metrics['communities']
)

mi_analysis = prepare_mutual_information_analysis(
    G, pos, metrics['node_metrics']
)

=== SPATIAL-ECOLOGICAL INFORMATION ANALYSIS ===

1. SPATIAL DISTRIBUTION PATTERNS:
   Spatial entropy (X): 3.310 bits
   Spatial entropy (Y): 3.318 bits
   Max possible spatial entropy: 3.322 bits
   Spatial uniformity: 0.998

2. ECOLOGICAL INFLUENCE PATTERNS:
   Influence entropy: -0.000 bits
   Max influence entropy: 2.322 bits
   Influence distribution uniformity: -0.000
   Mean influence: 0.000
   Std influence: 0.000
   Influence range: [0.000, 0.000]

3. COMMUNITY STRUCTURE INFORMATION:
   Number of communities: 9
   Community size entropy: 3.138 bits
   Community sizes: [120, 103, 101, 80, 75, 74, 72, 69, 64]
   Within-community edges: 6636 (0.811)
   Between-community edges: 1544 (0.189)
   Community connection entropy: 0.699 bits

4. INTERACTION SIGN INFORMATION:
   Positive edges: 7408 (0.906)
   Negative edges: 772 (0.094)
   Sign entropy: 0.451 bits
   Sign balance: 0.811

5. SPATIAL-ECOLOGICAL RELATIONSHIPS:
   Spatial-influence correlation: nan

6. HARVEST ENTROPY FOUNDAT

c:\Users\hp.LAPTOP-OG98VQR6\AppData\Local\Programs\Python\Python310\lib\site-packages\numpy\lib\function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
c:\Users\hp.LAPTOP-OG98VQR6\AppData\Local\Programs\Python\Python310\lib\site-packages\numpy\lib\function_base.py:2898: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]


In [ ]:
############################################################################################
# Check for reciprocal edges
############################################################################################

reciprocal_edges = []
unidirectional_edges = []

for u, v in G.edges():
    if G.has_edge(v, u):  # Check if reverse edge exists
        reciprocal_edges.append((u, v))
    else:
        unidirectional_edges.append((u, v))

print(f"Bidirectional edge pairs: {len(reciprocal_edges)//2}")  # Divide by 2 since each pair counted twice
print(f"Unidirectional edges: {len(unidirectional_edges)}")

Bidirectional edge pairs: 4090
Unidirectional edges: 0


In [19]:
def analyze_network_reciprocity(G):

    """
    Analyze the reciprocity patterns in your directed network
    """
    print("=== NETWORK RECIPROCITY ANALYSIS ===\n")
    
    total_edges = G.number_of_edges()
    total_possible_pairs = G.number_of_nodes() * (G.number_of_nodes() - 1) // 2
    
    # Count different types of relationships
    unidirectional_pairs = 0
    bidirectional_pairs = 0
    isolated_pairs = 0
    
    # Check all possible node pairs
    for u in G.nodes():
        for v in G.nodes():
            if u < v:  # Only check each pair once
                has_uv = G.has_edge(u, v)
                has_vu = G.has_edge(v, u)
                
                if has_uv and has_vu:
                    bidirectional_pairs += 1
                elif has_uv or has_vu:
                    unidirectional_pairs += 1
                else:
                    isolated_pairs += 1
    
    # Calculate reciprocity metrics
    connected_pairs = bidirectional_pairs + unidirectional_pairs
    
    if connected_pairs > 0:
        reciprocity_rate = bidirectional_pairs / connected_pairs
    else:
        reciprocity_rate = 0
    
    # NetworkX built-in reciprocity (different calculation)
    nx_reciprocity = nx.reciprocity(G)
    
    print(f"Total edges: {total_edges}")
    print(f"Total possible node pairs: {total_possible_pairs}")
    print(f"Connected pairs: {connected_pairs}")
    print(f"  - Bidirectional pairs: {bidirectional_pairs}")
    print(f"  - Unidirectional pairs: {unidirectional_pairs}")
    print(f"Isolated pairs: {isolated_pairs}")
    print(f"\nReciprocity rate: {reciprocity_rate:.3f}")
    print(f"NetworkX reciprocity: {nx_reciprocity:.3f}")
    
    # Sample some bidirectional relationships
    print(f"\nSample bidirectional relationships:")
    bidirectional_examples = []
    for u, v in G.edges():
        if G.has_edge(v, u) and len(bidirectional_examples) < 5:
            weight_uv = G[u][v]['weight']
            weight_vu = G[v][u]['weight']
            bidirectional_examples.append((u, v, weight_uv, weight_vu))
    
    for u, v, w_uv, w_vu in bidirectional_examples:
        print(f"  {u} ↔ {v}: weights {w_uv:.3f} and {w_vu:.3f}")
    
    # Sample some unidirectional relationships
    print(f"\nSample unidirectional relationships:")
    unidirectional_examples = []
    for u, v in G.edges():
        if not G.has_edge(v, u) and len(unidirectional_examples) < 5:
            weight_uv = G[u][v]['weight']
            unidirectional_examples.append((u, v, weight_uv))
    
    for u, v, w_uv in unidirectional_examples:
        print(f"  {u} → {v}: weight {w_uv:.3f}")
    
    return {
        'total_edges': total_edges,
        'bidirectional_pairs': bidirectional_pairs,
        'unidirectional_pairs': unidirectional_pairs,
        'reciprocity_rate': reciprocity_rate,
        'nx_reciprocity': nx_reciprocity
    }



In [ ]:
############################################################################################
### Analyze the network reciprocity
############################################################################################
# Run this analysis on your network:
reciprocity_stats = analyze_network_reciprocity(G)

In [20]:
############################################################################################
### Prepare mutual information analysis
############################################################################################

import numpy as np
from scipy.spatial.distance import pdist, squareform
from sklearn.metrics import mutual_info_score
from collections import Counter
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import networkx as nx  
import os
import glob

def analyze_spatial_information_structure(G, node_coords, node_metrics, communities, 
                                        spatial_bins=10, influence_bins=5):
    """
    Analyze the information structure of spatial ecological networks
    for future mutual information and harvest entropy calculations
    """
    
    print("=== SPATIAL-ECOLOGICAL INFORMATION ANALYSIS ===\n")
    
    # 1. SPATIAL DISTRIBUTION ANALYSIS
    print("1. SPATIAL DISTRIBUTION PATTERNS:")
    
    # Extract coordinates
    coords = np.array([node_coords[node] for node in G.nodes()])
    x_coords, y_coords = coords[:, 0], coords[:, 1]
    
    # Spatial entropy - how uniformly distributed are nodes in space?
    x_bins = np.histogram(x_coords, bins=spatial_bins)[0]
    y_bins = np.histogram(y_coords, bins=spatial_bins)[0]
    
    # Normalize to probabilities
    x_probs = x_bins[x_bins > 0] / x_bins.sum()
    y_probs = y_bins[y_bins > 0] / y_bins.sum()
    
    spatial_entropy_x = -np.sum(x_probs * np.log2(x_probs))
    spatial_entropy_y = -np.sum(y_probs * np.log2(y_probs))
    
    print(f"   Spatial entropy (X): {spatial_entropy_x:.3f} bits")
    print(f"   Spatial entropy (Y): {spatial_entropy_y:.3f} bits")
    print(f"   Max possible spatial entropy: {np.log2(spatial_bins):.3f} bits")
    print(f"   Spatial uniformity: {(spatial_entropy_x + spatial_entropy_y)/(2*np.log2(spatial_bins)):.3f}")
    
    # 2. ECOLOGICAL INFLUENCE DISTRIBUTION
    print("\n2. ECOLOGICAL INFLUENCE PATTERNS:")
    
    influences = []
    for node in G.nodes():
        if node in node_metrics and 'ecological_closeness' in node_metrics[node]:
            influence = node_metrics[node]['ecological_closeness'].get('total_influence', 0)
            influences.append(influence)
        else:
            influences.append(0)
    
    influences = np.array(influences)
    
    # Influence entropy
    influence_hist = np.histogram(influences, bins=influence_bins)[0]
    influence_probs = influence_hist[influence_hist > 0] / influence_hist.sum()
    influence_entropy = -np.sum(influence_probs * np.log2(influence_probs))
    
    print(f"   Influence entropy: {influence_entropy:.3f} bits")
    print(f"   Max influence entropy: {np.log2(influence_bins):.3f} bits")
    print(f"   Influence distribution uniformity: {influence_entropy/np.log2(influence_bins):.3f}")
    
    # Influence statistics
    print(f"   Mean influence: {np.mean(influences):.3f}")
    print(f"   Std influence: {np.std(influences):.3f}")
    print(f"   Influence range: [{np.min(influences):.3f}, {np.max(influences):.3f}]")
    
    # 3. COMMUNITY STRUCTURE ANALYSIS
    print("\n3. COMMUNITY STRUCTURE INFORMATION:")
    
    if communities and 'communities' in communities:
        # Community size distribution
        community_sizes = [len(nodes) for nodes in communities['communities'].values()]
        community_size_entropy = calculate_entropy_from_counts(community_sizes)
        
        print(f"   Number of communities: {len(communities['communities'])}")
        print(f"   Community size entropy: {community_size_entropy:.3f} bits")
        print(f"   Community sizes: {sorted(community_sizes, reverse=True)}")
        
        # Modularity-like measure: within vs between community connections
        within_community_edges = 0
        between_community_edges = 0
        
        # Create node-to-community mapping
        node_to_comm = {}
        for comm_id, nodes in communities['communities'].items():
            for node in nodes:
                node_to_comm[node] = comm_id
        
        for u, v in G.edges():
            if node_to_comm.get(u) == node_to_comm.get(v):
                within_community_edges += 1
            else:
                between_community_edges += 1
        
        total_edges = within_community_edges + between_community_edges
        if total_edges > 0:
            within_ratio = within_community_edges / total_edges
            between_ratio = between_community_edges / total_edges
            
            # Community connection entropy
            if within_ratio > 0 and between_ratio > 0:
                community_conn_entropy = -(within_ratio * np.log2(within_ratio) + 
                                         between_ratio * np.log2(between_ratio))
            else:
                community_conn_entropy = 0
            
            print(f"   Within-community edges: {within_community_edges} ({within_ratio:.3f})")
            print(f"   Between-community edges: {between_community_edges} ({between_ratio:.3f})")
            print(f"   Community connection entropy: {community_conn_entropy:.3f} bits")
    
    # 4. INTERACTION SIGN PATTERNS
    print("\n4. INTERACTION SIGN INFORMATION:")
    
    positive_edges = sum(1 for _, _, d in G.edges(data=True) if d['weight'] > 0)
    negative_edges = sum(1 for _, _, d in G.edges(data=True) if d['weight'] < 0)
    total_edges = positive_edges + negative_edges
    
    if total_edges > 0:
        pos_ratio = positive_edges / total_edges
        neg_ratio = negative_edges / total_edges
        
        # Sign entropy
        if pos_ratio > 0 and neg_ratio > 0:
            sign_entropy = -(pos_ratio * np.log2(pos_ratio) + neg_ratio * np.log2(neg_ratio))
        else:
            sign_entropy = 0
        
        print(f"   Positive edges: {positive_edges} ({pos_ratio:.3f})")
        print(f"   Negative edges: {negative_edges} ({neg_ratio:.3f})")
        print(f"   Sign entropy: {sign_entropy:.3f} bits")
        print(f"   Sign balance: {pos_ratio - neg_ratio:.3f}")
    
    # 5. SPATIAL-ECOLOGICAL CORRELATION PREPARATION
    print("\n5. SPATIAL-ECOLOGICAL RELATIONSHIPS:")
    
    # Prepare data for future mutual information analysis
    spatial_data = {
        'x_coords': x_coords,
        'y_coords': y_coords,
        'influences': influences,
        'distances_to_center': np.sqrt((x_coords - np.mean(x_coords))**2 + 
                                     (y_coords - np.mean(y_coords))**2)
    }
    
    # Spatial autocorrelation of influences
    # Calculate pairwise distances
    distances = squareform(pdist(coords))
    
    # Calculate influence correlation vs distance
    influence_pairs = []
    distance_pairs = []
    
    for i in range(len(influences)):
        for j in range(i+1, len(influences)):
            influence_pairs.append(influences[i] * influences[j])
            distance_pairs.append(distances[i, j])
    
    # Correlation between influence product and spatial distance
    if len(influence_pairs) > 1:
        spatial_influence_corr = np.corrcoef(influence_pairs, distance_pairs)[0, 1]
        print(f"   Spatial-influence correlation: {spatial_influence_corr:.3f}")
    
    # 6. HARVEST ENTROPY PREPARATION
    print("\n6. HARVEST ENTROPY FOUNDATIONS:")
    
    # Edge weight distribution (for future harvest calculations)
    weights = [abs(d['weight']) for _, _, d in G.edges(data=True)]
    if weights:
        weight_entropy = calculate_entropy_from_values(weights, bins=10)
        print(f"   Edge weight entropy: {weight_entropy:.3f} bits")
        print(f"   Mean edge weight: {np.mean(weights):.3f}")
        print(f"   Weight std: {np.std(weights):.3f}")
    
    # Node degree distribution (connectivity entropy)
    degrees = [G.degree(node) for node in G.nodes()]
    degree_entropy = calculate_entropy_from_counts(degrees)
    print(f"   Degree entropy: {degree_entropy:.3f} bits")
    
    return {
        'spatial_entropy': (spatial_entropy_x + spatial_entropy_y) / 2,
        'influence_entropy': influence_entropy,
        'community_structure': communities,
        'sign_entropy': sign_entropy if 'sign_entropy' in locals() else 0,
        'spatial_data': spatial_data,
        'network_stats': {
            'nodes': G.number_of_nodes(),
            'edges': G.number_of_edges(),
            'density': G.number_of_edges() / (G.number_of_nodes() * (G.number_of_nodes() - 1))
        }
    }

def calculate_entropy_from_counts(counts):
    """Calculate entropy from a list of counts"""
    counts = np.array(counts)
    counts = counts[counts > 0]  # Remove zeros
    probs = counts / counts.sum()
    return -np.sum(probs * np.log2(probs))

def calculate_entropy_from_values(values, bins=10):
    """Calculate entropy from continuous values by binning"""
    hist, _ = np.histogram(values, bins=bins)
    return calculate_entropy_from_counts(hist)

def prepare_mutual_information_analysis(G, node_coords, node_metrics, spatial_bins=5):
    """
    Prepare discretized data for mutual information calculations
    """
    print("\n=== MUTUAL INFORMATION PREPARATION ===")
    
    # Discretize spatial coordinates
    coords = np.array([node_coords[node] for node in G.nodes()])
    x_coords, y_coords = coords[:, 0], coords[:, 1]
    
    # Create spatial bins
    x_digitized = np.digitize(x_coords, np.linspace(x_coords.min(), x_coords.max(), spatial_bins))
    y_digitized = np.digitize(y_coords, np.linspace(y_coords.min(), y_coords.max(), spatial_bins))
    
    # Discretize influences
    influences = []
    for node in G.nodes():
        if node in node_metrics and 'ecological_closeness' in node_metrics[node]:
            influence = node_metrics[node]['ecological_closeness'].get('total_influence', 0)
            influences.append(influence)
        else:
            influences.append(0)
    
    influences = np.array(influences)
    influence_digitized = np.digitize(influences, np.linspace(influences.min(), influences.max(), spatial_bins))
    
    # Calculate mutual information between spatial and ecological variables
    mi_x_influence = mutual_info_score(x_digitized, influence_digitized)
    mi_y_influence = mutual_info_score(y_digitized, influence_digitized)
    mi_xy_spatial = mutual_info_score(x_digitized, y_digitized)
    
    print(f"Mutual Information (X-position, Influence): {mi_x_influence:.3f} bits")
    print(f"Mutual Information (Y-position, Influence): {mi_y_influence:.3f} bits")
    print(f"Mutual Information (X-position, Y-position): {mi_xy_spatial:.3f} bits")
    
    return {
        'spatial_discretized': (x_digitized, y_digitized),
        'influence_discretized': influence_digitized,
        'mutual_info': {
            'x_influence': mi_x_influence,
            'y_influence': mi_y_influence,
            'xy_spatial': mi_xy_spatial
        }
    }

In [ ]:
############################################
###DEBUGGED with ChatGPT
###ChatGPT-04-mini-high###
############################################
### XY Netorks with Entropy calculations

In [ ]:
############################################################################################
### XY Netorks with Entropy calculations
############################################################################################

import numpy as np
from scipy.spatial.distance import pdist, squareform
from sklearn.metrics import mutual_info_score
from collections import Counter
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import networkx as nx  
import os
import glob

data_dir = r"C:\Users\hp.LAPTOP-OG98VQR6\OneDrive\Desktop\ERSA\Agroforestal\Discover Networks\Simulations\Simulation6"
# data_dir = r"C:\Users\hp.LAPTOP-OG98VQR6\OneDrive\Desktop\ERSA\Agroforestal\Discover Networks\Simulations\Simulation3"
# data_dir = r"C:\Users\hp.LAPTOP-OG98VQR6\OneDrive\Desktop\ERSA\Agroforestal\Discover Networks\Simulations\Simulation4
# data_dir = r"C:\Users\hp.LAPTOP-OG98VQR6\OneDrive\Desktop\ERSA\Agroforestal\Discover Networks\Simulations\Simulation5"
pattern = os.path.join(data_dir, "weighted_matrix_with_coords_combined_tick_*.csv")
files = glob.glob(pattern)
# sort by tick-number at end of filename
files = sorted(files, key=lambda fp: int(os.path.basename(fp).split('_')[-1].split('.')[0]))

ticks = []
entropies = []

for fp in files:
    # 1) extract tick
    tick = int(os.path.basename(fp).split('_')[-1].split('.')[0])
    ticks.append(tick)

    # 2) read csv
    df = pd.read_csv(fp)
    
    # 3) compute entropy
    # pivot to adjacency
    adj = df.pivot(index='source', columns='target', values='weight').fillna(0)
    P   = adj.div(adj.sum(axis=1), axis=0)
    p   = P.values.flatten()
    p   = p[p>0]
    p  /= p.sum()
    H   = -(p * np.log2(p)).sum()
    entropies.append(H)
    print(f"tick {tick:3d}: H = {H:.4f} bits")

    # 4) build graph
    G = nx.DiGraph()
    for _, row in df.iterrows():
        u, v, w = int(row.source), int(row.target), float(row.weight)
        G.add_edge(u, v, weight=w)
    # 5) build pos dict from your source_x/source_y and target_x/target_y
    coord_rows = pd.concat([
        df[['source','source_x','source_y']].rename(columns={'source':'node','source_x':'x','source_y':'y'}),
        df[['target','target_x','target_y']].rename(columns={'target':'node','target_x':'x','target_y':'y'})
    ]).drop_duplicates(subset='node')
    pos = { int(r.node): (r.x, r.y) for r in coord_rows.itertuples() }

    # 6) visualize at true coordinates
    fig, ax = plt.subplots(figsize=(8,8))
    # edge widths/colors
    edges = G.edges(data=True)
    weights = [abs(d['weight'])*3 for _,_,d in edges]
    colors  = [      d['weight']   for _,_,d in edges]
    # nodes
    nx.draw_networkx_nodes(G, pos, ax=ax, node_size=30, node_color='k', alpha=0.7)
    # edges
    ec = nx.draw_networkx_edges(
        G, pos, ax=ax,
        arrowstyle='-',
        width=weights,
        edge_color=colors,
        edge_cmap=plt.cm.RdBu_r,
        edge_vmin=-max(abs(min(colors)),abs(max(colors))),
        edge_vmax= max(abs(min(colors)),abs(max(colors))),
        alpha=0.6
    )
    # colorbar
    sm = plt.cm.ScalarMappable(
        cmap=plt.cm.RdBu_r,
        norm=plt.Normalize(
            vmin=-max(abs(min(colors)),abs(max(colors))),
             vmax= max(abs(min(colors)),abs(max(colors)))
        )
    )
    sm.set_array([])
    fig.colorbar(sm, ax=ax, label='edge weight')
    ax.set_title(f"Tick {tick}  —  H={H:.3f} bits")
    ax.set_aspect('equal')
    ax.axis('off')
    plt.tight_layout()
    plt.savefig(f"network_tick_{tick:03d}.png", dpi=300)
    plt.close(fig)

# 7) finally plot entropy series
plt.figure(figsize=(6,3))
plt.plot(ticks, entropies, '-o')
plt.xlabel('Tick')
plt.ylabel('Shannon entropy (bits)')
plt.tight_layout()
plt.show()


In [ ]:
############################################################################################
### XY Netorks with Entropy calculations Edit - 1
############################################################################################

import numpy as np
from scipy.spatial.distance import pdist, squareform
from sklearn.metrics import mutual_info_score
from collections import Counter
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import networkx as nx  
import os
import glob


# 1) find & sort your files
#data_dir    = r"C:\Users\hp.LAPTOP-OG98VQR6\OneDrive\Desktop\ERSA\Agroforestal\Discover Networks\Simulations\Simulation5"
data_dir   = r"C:\Users\hp.LAPTOP-OG98VQR6\OneDrive\Desktop\ERSA\Agroforestal\Discover Networks\Simulations\Simulation6"

pattern     = os.path.join(data_dir, "weighted_matrix_with_coords_combined_tick_*.csv")
files       = glob.glob(pattern)
files.sort(key=lambda fp: int(os.path.basename(fp).split('_')[-1].split('.')[0]))

# helper to pull tick number out of filename
def extract_tick(fp):
    return int(os.path.basename(fp).split('_')[-1].split('.')[0])

# the core loop
ticks      = []
entropies  = []

for fp in files:
    tick = extract_tick(fp)
    ticks.append(tick)

    # --- read & entropy ---
    df = pd.read_csv(fp)
    adj = df.pivot(index='source', columns='target', values='weight').fillna(0)
    P   = adj.div(adj.sum(axis=1), axis=0)
    p   = P.values.flatten()
    p   = p[p > 0]
    p  /= p.sum()
    H   = -(p * np.log2(p)).sum()
    entropies.append(H)
    print(f"tick {tick:3d}: H = {H:.4f} bits")

    # --- build graph & extract real positions ---
    G = nx.DiGraph()
    pos = {}
    for _, row in df.iterrows():
        src, tgt = int(row.source), int(row.target)
        w        = row.weight

        # add nodes + record their true x/y
        if src not in G:
            G.add_node(src)
            pos[src] = (row.source_x, row.source_y)
        if tgt not in G:
            G.add_node(tgt)
            pos[tgt] = (row.target_x, row.target_y)

        # add the directed edge
        G.add_edge(src, tgt, weight=w)

    # --- visualize with reversed cmap ---
    fig, ax = plt.subplots(figsize=(8, 6))
    nx.draw_networkx_nodes(
        G, pos, ax=ax,
        node_size=50,
        node_color='green',
        alpha=0.8
    )

    # edge styling
    weights = [abs(d['weight']) * 2 for _,_,d in G.edges(data=True)]
    colors  = [d['weight']        for _,_,d in G.edges(data=True)]
    M       = max(abs(min(colors)), abs(max(colors)))

    nx.draw_networkx_edges(
        G, pos, ax=ax,
        width=weights,
        edge_color=colors,
        edge_cmap=plt.cm.RdBu_r,    # reversed RdBu
        edge_vmin=-M,
        edge_vmax=+M,
        alpha=0.6
    )

    sm = plt.cm.ScalarMappable(
        cmap=plt.cm.RdBu_r,
        norm=plt.Normalize(vmin=-M, vmax=+M)
    )
    sm.set_array([])
    cbar = fig.colorbar(sm, ax=ax)
    cbar.set_label("Edge weight")

    ax.set_title(f"Network at tick {tick}")
    ax.axis('off')
    plt.tight_layout()
    outname = f"network_tick_{tick:03d}.png"
    plt.savefig(outname, dpi=300)
    plt.close(fig)

# --- final H vs tick plot ---
plt.figure(figsize=(6,4))
plt.plot(ticks, entropies, '-o')
plt.xlabel("Tick")
plt.ylabel("Shannon entropy (bits)")
plt.tight_layout()
plt.savefig("entropy_vs_tick.png", dpi=300)
plt.show()


In [ ]:
############################################################################################
### XY Netorks with Entropy calculations Edit - 2
############################################################################################

import numpy as np
from scipy.spatial.distance import pdist, squareform
from sklearn.metrics import mutual_info_score
from collections import Counter
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import networkx as nx  
import os
import glob


data_dir    = r"C:\Users\hp.LAPTOP-OG98VQR6\OneDrive\Desktop\ERSA\Agroforestal\Discover Networks\Simulations\Simulation6"
# data_dir    = r"C:\Users\hp.LAPTOP-OG98VQR6\OneDrive\Desktop\ERSA\Agroforestal\Discover Networks\Simulations\Simulation5"

pattern = os.path.join(data_dir, "weighted_matrix_with_coords_combined_tick_*.csv")
files = glob.glob(pattern)
# sort by tick-number at end of filename
files = sorted(files, key=lambda fp: int(os.path.basename(fp).split('_')[-1].split('.')[0]))

# Define the output directory for network plots
output_dir = os.path.join(data_dir, "network_plots\diagnostic_plots")
os.makedirs(output_dir, exist_ok=True)  # Create the directory if it doesn't exist

# the core loop
ticks      = []
entropies  = []

for fp in files:
    # 1) extract tick
    tick = int(os.path.basename(fp).split('_')[-1].split('.')[0])
    ticks.append(tick)

    # 2) read csv
    df = pd.read_csv(fp)
    
    # 3) compute entropy
    # pivot to adjacency
    adj = df.pivot(index='source', columns='target', values='weight').fillna(0)
    P   = adj.div(adj.sum(axis=1), axis=0)
    p   = P.values.flatten()
    p   = p[p>0]
    p  /= p.sum()
    H   = -(p * np.log2(p)).sum()
    entropies.append(H)
    print(f"tick {tick:3d}: H = {H:.4f} bits")

    # 4) build graph
    G = nx.DiGraph()
    for _, row in df.iterrows():
        u, v, w = int(row.source), int(row.target), float(row.weight)
        G.add_edge(u, v, weight=w)
    # 5) build pos dict from your source_x/source_y and target_x/target_y
    coord_rows = pd.concat([
        df[['source','source_x','source_y']].rename(columns={'source':'node','source_x':'x','source_y':'y'}),
        df[['target','target_x','target_y']].rename(columns={'target':'node','target_x':'x','target_y':'y'})
    ]).drop_duplicates(subset='node')
    pos = { int(r.node): (r.x, r.y) for r in coord_rows.itertuples() }

    # 6) visualize at true coordinates
    fig, ax = plt.subplots(figsize=(8,8))
    # edge widths/colors
    # 6) visualize at true coordinates
    edges = G.edges(data=True)
    weights = [abs(d['weight'])*3 for _,_,d in edges]
    colors  = [      d['weight']   for _,_,d in edges]

    # nodes - changed to green
    nx.draw_networkx_nodes(G, pos, ax=ax, node_size=30, node_color='green', alpha=0.7)

    # edges - inverted colormap: negative edges = red, positive edges = blue
    ec = nx.draw_networkx_edges(
        G, pos, ax=ax,
        arrowstyle='-',
        width=weights,
        edge_color=colors,
        edge_cmap=plt.cm.RdBu,    # _r reverses the colormap: red for negative, blue for positive
        edge_vmin=-max(abs(min(colors)),abs(max(colors))),
        edge_vmax= max(abs(min(colors)),abs(max(colors))),
        alpha=0.6
    )

    # colorbar - also use reversed colormap
    sm = plt.cm.ScalarMappable(
        cmap=plt.cm.RdBu,  # _r to match the edges
        norm=plt.Normalize(
            vmin=-max(abs(min(colors)),abs(max(colors))),
            vmax= max(abs(min(colors)),abs(max(colors)))
        )
    )

    sm.set_array([])
    fig.colorbar(sm, ax=ax, label='edge weight')
    ax.set_title(f"Tick {tick}  —  H={H:.3f} bits")
    ax.set_aspect('equal')
    ax.axis('off')
    plt.tight_layout()
    outpath = os.path.join(output_dir, f"network_tick_{tick:03d}.png")
    plt.savefig(outpath, dpi=300)
    print(f"  → saved {outpath}")
    plt.close(fig)
# --- final H vs tick plot ---
plt.figure(figsize=(6,4))
plt.plot(ticks, entropies, '-o')
plt.xlabel("Tick")
plt.ylabel("Shannon entropy (bits)")
plt.tight_layout()
plt.savefig(os.path.join(output_dir, "entropy_vs_tick.png"), dpi=300)
plt.show()

In [ ]:
#############################################################################################
### XY Netorks ECOLOGICAL METRICS & VISUALIZATION - FINAL
#############################################################################################

import os, glob
import pandas as pd
import matplotlib.pyplot as plt

# --- paths ---
data_dir = r"C:\Users\hp.LAPTOP-OG98VQR6\OneDrive\Desktop\ERSA\Agroforestal\Discover Networks\Simulations\Simulation6"
pattern   = os.path.join(data_dir, "weighted_matrix_with_coords_combined_tick_*.csv")

# collect & sort by tick number
files = sorted(
    glob.glob(pattern),
    key=lambda fp: int(os.path.basename(fp).split('_')[-1].split('.')[0])
)

# output dir
output_dir = os.path.join(data_dir, "network_graphs")
os.makedirs(output_dir, exist_ok=True)

def extract_tick(fp):
    return int(os.path.basename(fp).split('_')[-1].split('.')[0])

ticks = []

for fp in files:
    try:
        # 1) tick
        tick = extract_tick(fp)
        ticks.append(tick)

        # 2) read this file's CSV ONCE
        df = pd.read_csv(fp)

        # 3) build graph for THIS file (important: use fp, not file_path)
        G = load_network_from_csv(fp)

        # 4) node coordinates for THIS file
        coord_rows = pd.concat([
            df[['source','source_x','source_y']].rename(
                columns={'source':'node','source_x':'x','source_y':'y'}
            ),
            df[['target','target_x','target_y']].rename(
                columns={'target':'node','target_x':'x','target_y':'y'}
            ),
        ], ignore_index=True).drop_duplicates(subset='node')

        # ensure node ids line up with G; if your nodes are ints, cast:
        try:
            coord_rows['node'] = coord_rows['node'].astype(int)
        except Exception:
            pass

        pos = {row['node']: (row['x'], row['y']) for _, row in coord_rows.iterrows()}

        # 5) metrics
        metrics = analyze_direct_and_indirect_interactions(G)
        print(summarize_ecological_metrics(metrics))

        # 6) draw & save
        fig, ax = visualize_spatial_network(
        G, pos,
        node_metrics=metrics.get('node_metrics', {}),
        communities=metrics.get('communities', {}),
        title=f"Spatial Ecological Network – tick {tick}",
        edge_width_factor=2.0,
        node_size_factor=50,
        show_labels=True
    )

        outname = os.path.join(output_dir, f"Spatial_Ecological_Network_{tick:03d}.png")
        fig.savefig(outname, dpi=300, bbox_inches='tight')
        plt.close(fig)  # important to avoid memory bloat in loops
        print(f"  Saved: {outname}")

        # (optional) briefly show
        from IPython.display import display
        display(fig)

    except Exception as e:
        print(f"[WARN] Skipping tick {tick} due to error: {e}")


In [ ]:
#############################################################################################
### XY Netorks ECOLOGICAL METRICS & VISUALIZATION - FINAL 2
#############################################################################################

import os, glob, sys
import pandas as pd
import matplotlib.pyplot as plt

# ---------- config ----------
data_dir   = r"C:\Users\hp.LAPTOP-OG98VQR6\OneDrive\Desktop\ERSA\Agroforestal\Discover Networks\Simulations\Simulation6"
pattern    = os.path.join(data_dir, "weighted_matrix_with_coords_combined_tick_*.csv")
output_dir = os.path.join(data_dir, "network_graphs")
os.makedirs(output_dir, exist_ok=True)

def extract_tick(fp):
    try:
        return int(os.path.basename(fp).split('_')[-1].split('.')[0])
    except Exception:
        return None

# ---------- discover files ----------
files = glob.glob(pattern)
print(f"[INFO] Searching: {pattern}")
print(f"[INFO] Found {len(files)} files")

# Show a few examples so we can see the naming
for f in sorted(files)[:5]:
    print("  ->", os.path.basename(f))

# If nothing found, stop here
if not files:
    print("[ERROR] No files matched the pattern. Check data_dir/pattern.")
    sys.exit(1)

# Sort by tick; skip any file we can't parse
files = sorted(
    (f for f in files if extract_tick(f) is not None),
    key=lambda fp: extract_tick(fp)
)
print(f"[INFO] After sorting by tick: {len(files)} files")

# ---------- main loop ----------
for fp in files:
    tick = extract_tick(fp)
    print(f"\n[INFO] Processing tick {tick}: {os.path.basename(fp)}")

    # 1) Read once and check columns
    df = pd.read_csv(fp)
    print(f"[INFO] CSV rows: {len(df)} | columns: {list(df.columns)}")

    needed = ['source','target','weight','source_x','source_y','target_x','target_y']
    missing = [c for c in needed if c not in df.columns]
    if missing:
        print(f"[ERROR] Missing columns in {os.path.basename(fp)}: {missing}")
        continue

    # 2) Build graph for THIS file
    G = load_network_from_csv(fp)  # this should print its own summary; if it doesn't, we didn't call it
    # 3) Position dict
    coords = pd.concat([
        df[['source','source_x','source_y']].rename(columns={'source':'node','source_x':'x','source_y':'y'}),
        df[['target','target_x','target_y']].rename(columns={'target':'node','target_x':'x','target_y':'y'})
    ], ignore_index=True).drop_duplicates(subset='node')

    try:
        coords['node'] = coords['node'].astype(int)
    except Exception:
        pass
    pos = {int(r.node): (float(r.x), float(r.y)) for _, r in coords.iterrows()}

    # 4) Metrics
    metrics = analyze_direct_and_indirect_interactions(G)
    print(summarize_ecological_metrics(metrics))

    # 5) Draw
    fig, ax = visualize_spatial_network(
    G, pos,
    node_metrics=metrics.get('node_metrics', {}),
    communities=metrics.get('communities', {}),
    title=f"Spatial Ecological Network – tick {tick}",
    edge_width_factor=2.0,
    node_size_factor=50,
    show_labels=True
)

    if fig is None:
        print("[ERROR] visualize_spatial_network returned None. "
              "Ensure it RETURNS the matplotlib Figure and does NOT call plt.close() inside.")
        continue

    # 6) Save THEN (optionally) show
    outname = os.path.join(output_dir, f"Spatial_Ecological_Network_{tick:03d}.png")
    fig.savefig(outname, dpi=300, bbox_inches='tight')
    print(f"[OK] Saved: {outname}")

    outname = os.path.join(output_dir, f"Spatial_Ecological_Network_{tick:03d}.png")
    fig.savefig(outname, dpi=300, bbox_inches='tight')
    plt.close(fig)
    print(f"[OK] Saved: {outname}")

    # If you want to see the figure interactively, show BEFORE closing:
    # plt.show(block=False); plt.pause(0.5)

    plt.close(fig)  # free memory


In [24]:
import pandas as pd, os
# If df_ts is already in memory, use it; otherwise read from disk
try:
    df = df.copy()
except NameError:
    df = pd.read_csv(os.path.join(output_dir, 'all_metrics.csv'), index_col=0)

# Check which columns are actually present in the dataframe
available_cols = [col for col in [
    'degree_entropy','weighted_degree_entropy','community_entropy','num_communities','modularity',
    'kl_weight_vs_harvest','js_weight_vs_harvest','harvest_entropy','pearson_w_h',
    'js_boot_mean','js_boot_lo','js_boot_hi'
] if col in df.columns]

if not available_cols:
    print("None of the requested columns are present in the dataframe. Available columns are:")
    print(df.columns.tolist())
else:
    sel = df.loc[[52,182,208,234,260,390], available_cols]
    print(sel.round(9))
    # Optional: export a small CSV for the manuscript folder
    sel.round(9).to_csv(os.path.join(output_dir, 'selected_metrics_ticks_52_182_208_234_260_390.csv'))

None of the requested columns are present in the dataframe. Available columns are:
['source', 'target', 'weight', 'type', 'link-interaction-type', 'source_x', 'source_y', 'target_x', 'target_y']
